# NPC cancer recurrece Detection with ML
> we will train the model of NPC cancer recurrece detection.logistic regression and random forest will be used.

- author: Nityanantan
- categories: [Python, Machine_Learning]



##Import library

In [ ]:
import sys
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from pandas.plotting import scatter_matrix
import sklearn
import seaborn as sns


from sklearn import preprocessing
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from xgboost import XGBClassifier

from google.colab import files
from math import isnan

import pickle

## Read data and explore

In [ ]:
df = pd.read_csv('Data_combined.csv')

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df.astype('object').describe().transpose()

## Cleaning

In [ ]:
#https://www.qxmd.com/calculate/calculator_620/nasopharyngeal-cancer-tnm-staging
#tnm staging level for NPC

def tnm(T,N,M,stage):
	if ((T == "2")and(N == "0")and(M == "0")) :
		return "0"
	elif ((T == "1")and(N == "0")and(M == "0")) :
		return "I"
	elif ((T == "1") or (T == "0")and(N == "1")and(M == "0")):
		return "II"
	elif ((T == "2")and(N == "0")and(M == "0")) :
		return "II"
	elif ((T == "2")and(N == "1")and(M == "0")) :
		return "II"
	elif ((T == "1") or (T == "0")and(N == "2")and(M == "0")):
		return "III"
	elif ((T == "2")and(N == "2")and(M == "0")):
		return "III"
	elif ((T == "3")and((N == "0") or (N == "1") or (N == "2"))and(M == "0")):
		return "III"
	elif ((T == "4")and((N == "0") or (N == "1") or (N == "2"))and(M == "0")):
		return "IVA"
	elif (((T == "0") or (T == "0") or (T == "2") or (T == "1") or (T == "2") or (T == "3") or (T == "4"))and(N == "3")and(M == "0")):
		return "IVA"
	elif (((T == "0") or (T == "0") or (T == "2") or (T == "1") or (T == "2") or (T == "3") or (T == "4"))and((N == "0") or (N == "0") or (N == "1") or (N == "2") or (N == "3"))and(M == "1")):
		return "IVB"
	return stage

In [ ]:
#clean tnm stage unknown
df['Stage'] = df.apply(lambda row: np.nan if row['Stage'] == 'unknown' else row['Stage'],axis=1)

In [ ]:
#clean tnm stage
df['Stage'] = df.apply(lambda row: tnm(row['T'],row['N'],row['M'],row['Stage']) if row['Stage'] is np.nan else row['Stage'],axis=1)

In [ ]:
#missing value analysis

mis_val = df.isnull().sum()
# Percentage of missing values
mis_val_percent = 100 * df.isnull().sum() / len(df)
# Make a table with the results
mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
# Rename the columns
mis_val_table_ren_columns = mis_val_table.rename(
columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
# Sort the table by percentage of missing descending
mis_val_table_ren_columns = mis_val_table_ren_columns[
    mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
    '% of Total Values', ascending=False).round(1)
        
# Print some summary information
print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"  +
       "There are " + str(mis_val_table_ren_columns.shape[0]) +
       " columns that have missing values.")
# Missing values for training data
mis_val_table_ren_columns[:20].style.background_gradient(cmap='Blues')

In [ ]:
#create target variable from treatement outcome
def categorise(row):  
    if row['TreatmentOutcome'] == 'Recurrence':
        return 1
    return 0

df['Recurrence'] = df.apply(lambda row: categorise(row), axis=1)


In [ ]:
# Drop useless feature
df=df.drop(['Age_At_Last_Status_Update','Status','FirstPositiveBiopsyDate','Hospital','RT_EndDate','Days_RTEnd_untill_Death','Days_Diag_untill_Death','DeathDate','CT_EndDate','CT_Started_After','DOB','StatusDate','ENT_1st_Visit','Onco_1st_Visit','Critical_Level','T','N','M','TreatmentOutcome','Ethnicity','Occupation','PatientID','Name'], axis=1)

In [ ]:
# Drop high null values column
df=df.drop(['SurgeriesCount'], axis=1)

In [ ]:
#imputing values in null space

def lin_regression_impute(df, model):
        # function for predicting missing values with linear regression
        cols_num = df.select_dtypes(include=np.number).columns
        mapping = dict()
        for feature in df.columns:
            if feature not in cols_num:
                # create label mapping for categorical feature values
                mappings = {k: i for i, k in enumerate(df[feature])}
                mapping[feature] = mappings
                df[feature] = df[feature].map(mapping[feature])
        for feature in cols_num: 
                try:
                    test_df = df[df[feature].isnull()==True].dropna(subset=[x for x in df.columns if x != feature])
                    train_df = df[df[feature].isnull()==False].dropna(subset=[x for x in df.columns if x != feature])
                    if len(test_df.index) != 0:
                        pipe = make_pipeline(StandardScaler(), model)

                        y = np.log(train_df[feature]) # log-transform the data
                        X_train = train_df.drop(feature, axis=1)
                        test_df.drop(feature, axis=1, inplace=True)
                        
                        try:
                            model = pipe.fit(X_train, y)
                        except:
                            y = train_df[feature] # use non-log-transformed data
                            model = pipe.fit(X_train, y)
                        if (y == train_df[feature]).all():
                            pred = model.predict(test_df)
                        else:
                            pred = np.exp(model.predict(test_df)) # predict values

                        test_df[feature]= pred

                        if (df[feature].fillna(-9999) % 1  == 0).all():
                            # round back to INTs, if original data were INTs
                            test_df[feature] = test_df[feature].round()
                            test_df[feature] = test_df[feature].astype('Int64')
                            df[feature].update(test_df[feature])                          
                        else:
                            df[feature].update(test_df[feature])  
                        print('LINREG imputation succeeded', len(pred), feature)
                except:
                    print('LINREG imputation failed', feature)
        for feature in df.columns: 
            try:   
                # map categorical feature values back to original
                mappings_inv = {v: k for k, v in mapping[feature].items()}
                df[feature] = df[feature].map(mappings_inv)
            except:
                pass
        return df





def impute(df, imputer, type):
        # function for imputing missing values in the data
        cols_num = df.select_dtypes(include=np.number).columns 

        if type == 'num':
            # numerical features
            for feature in df.columns: 
                if feature in cols_num:
                    if df[feature].isna().sum().sum() != 0:
                        try:
                            df_imputed = pd.DataFrame(imputer.fit_transform(np.array(df[feature]).reshape(-1, 1)))
                            counter = df[feature].isna().sum().sum() - df_imputed.isna().sum().sum()

                            if (df[feature].fillna(-9999) % 1  == 0).all():
                                df[feature] = df_imputed
                                # round back to INTs, if original data were INTs
                                df[feature] = df[feature].round()
                                df[feature] = df[feature].astype('Int64')                                        
                            else:
                                df[feature] = df_imputed
                            if counter != 0:
                                print('imputation succeeded', str(counter, feature))
                        except:
                            print('imputation failed', str(feature))
        else:
            # categorical features
            for feature in df.columns:
                if feature not in cols_num:
                    if df[feature].isna().sum()!= 0:
                        try:
                            mapping = dict()
                            mappings = {k: i for i, k in enumerate(df[feature].dropna().unique(), 0)}
                            mapping[feature] = mappings
                            df[feature] = df[feature].map(mapping[feature])

                            df_imputed = pd.DataFrame(imputer.fit_transform(np.array(df[feature]).reshape(-1, 1)), columns=[feature])    
                            counter = sum(1 for i, j in zip(list(df_imputed[feature]), list(df[feature])) if i != j)

                            # round to integers before mapping back to original values
                            df[feature] = df_imputed
                            df[feature] = df[feature].round()
                            df[feature] = df[feature].astype('Int64')  

                            # map values back to original
                            mappings_inv = {v: k for k, v in mapping[feature].items()}
                            df[feature] = df[feature].map(mappings_inv)
                            if counter != 0:
                                print('imputation succeeded', counter, feature)
                        except:
                            print('imputation failed', str( feature))
        return df

In [ ]:
# immpute values with linear regression
lr = LinearRegression()
df = lin_regression_impute(df, lr)

In [ ]:
# immpute values with KNN
imputer = KNNImputer(n_neighbors=3)
df = impute(df, imputer, type='cat')

In [ ]:
imputer = KNNImputer(n_neighbors=3)
df = impute(df, imputer, type='num')

In [ ]:
#encoding
def to_onehot(df, feature, limit=10):  
        # function that encodes categorical features to OneHot encodings    
        one_hot = pd.get_dummies(df[feature], prefix=feature)
        if one_hot.shape[1] > limit:
            print( feature, one_hot.shape[1])
        # join the encoded df
        df = df.join(one_hot)
        return df

def to_label(df, feature):
        # function that encodes categorical features to label encodings 
        le = preprocessing.LabelEncoder()

        df[feature] = le.fit_transform(df[feature].values)
        mapping = dict(zip(le.classes_, range(len(le.classes_))))
        
        for key in mapping:
            try:
                if isnan(key):               
                    replace = {mapping[key] : key }
                    df[feature].replace(replace, inplace=True)
            except:
                pass
        return df  


cols_categ = set(df.columns) ^ set(df.select_dtypes(include=np.number).columns) 
target_cols = cols_categ # encode ALL columns
for feature in target_cols:
  if df[feature].nunique() <=10:
    df = to_onehot(df, feature)
    print('Encoding to ONEHOT succeeded', feature)
    df=df.drop([feature],axis=1)
  elif df[feature].nunique() <=20:
    df = to_label(df, feature)
    print('Encoding to LABEL succeeded', feature)


##Modelling

In [ ]:
scaler = StandardScaler().fit(df.drop("Recurrence",axis=1))


In [ ]:
X = scaler.transform(df.drop("Recurrence",axis=1))
y = df["Recurrence"]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.20,random_state=0)

In [ ]:
from sklearn.metrics import plot_roc_curve,plot_confusion_matrix,accuracy_score,confusion_matrix
def Model(model):
    global X,y,X_train, X_test, y_train, y_test
    print(type(model).__name__)
    pred = model.predict(X_test)
    acs = accuracy_score(y_test,pred)
    print("Accuracy Score             :",acs)

In [ ]:
from sklearn.model_selection import cross_val_score
def CrossValidationScore(model_list):
    global X,y
    
    mean_cross_val_score = []
    model_name           = []
    
    for model in model_list:
        model_name.append(type(model).__name__)
        
    for i in model_list:
        scores = cross_val_score(i, X, y, cv=5)
        mean_cross_val_score.append(scores.mean())
        
    cvs = pd.DataFrame({"Model Name":model_name,"CVS":mean_cross_val_score})
    return cvs.style.background_gradient("Greens")

In [ ]:
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

In [ ]:
from sklearn.model_selection import cross_val_score
dtc = DecisionTreeClassifier()
lr = LogisticRegression()
svm = SVC()
knc = KNeighborsClassifier()
rfc = RandomForestClassifier()
gbc = GradientBoostingClassifier()
abc = AdaBoostClassifier()

keys = ['DecisionTreeClassifier','LogisticRegression','KNeighborsClassifier','RandomForestClassifier','GradientBoostingClassifier','AdaBoostClassifier','XGBClassifier']
values = [DecisionTreeClassifier(), LogisticRegression(), KNeighborsClassifier(), RandomForestClassifier(), GradientBoostingClassifier(), AdaBoostClassifier(),XGBClassifier()]

scores = []
for value in values:
 Model(value)

scores

In [ ]:
from sklearn.metrics import confusion_matrix , classification_report
model= RandomForestClassifier(n_estimators=100, criterion-'entropy', random_State=0)
model.fit(X_train,y_train)
predict=model.predict(X_test)
print(classification_report(y_test,predict))
print(mode.score(X_test,y_test))

In [ ]:
#compare models
plt.figure(figsize = (10,5))
sns.barplot(x = scores, y = keys, palette='muted')
plt.title("Model Scores", fontsize=16, fontweight="bold")

## Dump pickle model file

In [ ]:
pickle.dump(model, open('rf_model.pkl', 'wb'))
